In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
from gaze_tracking import GazeTracking

%matplotlib inline
m=6
cam_index=0
initialize_time=20 #change by observing the cam detection time
cam_setection_time=16
k=cam_index


def high(img):
    #img = cv2.resize(img,(int(img.shape[1]*m),int(img.shape[0]*m)))
    img = cv2.resize(img,(240,100))
    dst = cv2.fastNlMeansDenoising(img,None,200.0, 7, 21)
    #dst2 = cv2.fastNlMeansDenoising(dst,None,20.0,7,21)
    
    #dst3 = cv2.fastNlMeansDenoising(dst2,None,50,7,21)
    '''
    dst4 = cv2.fastNlMeansDenoisingColored(dst3,None,10,10,7,21)
    dst5 = cv2.fastNlMeansDenoisingColored(dst4,None,10,10,7,21)
    dst6 = cv2.fastNlMeansDenoisingColored(dst5,None,10,10,7,21)
    dst7 = cv2.fastNlMeansDenoisingColored(dst6,None,10,10,7,21)
    dst8 = cv2.fastNlMeansDenoisingColored(dst7,None,10,10,7,21)
    dst9 = cv2.fastNlMeansDenoisingColored(dst8,None,10,10,7,21)
    dst10 = cv2.fastNlMeansDenoisingColored(dst9,None,10,10,7,21)'''

    return dst #RGB    

def blob(im,center):
    params = cv2.SimpleBlobDetector_Params()
    params.filterByArea = True
    params.minArea = 270  # The dot in 20pt font has area of about 30
    params.filterByColor = 1
    #params.blobColor = 255
    params.filterByCircularity = 0
    params.filterByConvexity = 0
    params.filterByInertia = 0
    detector = cv2.SimpleBlobDetector_create(params)
    keypoints = detector.detect(im)
    im_with_keypoints=cv2.cvtColor(im,cv2.COLOR_GRAY2BGR)
    try:
        pt1=im.shape[1]//2-keypoints[0].pt[0]
        pt2=im.shape[1]//2-keypoints[0].pt[1]
        my=cv2.KeyPoint(im.shape[1]//2,im.shape[0]//2,0.2)
    except IndexError:pass
    #(100,180)
    #cv2.putText(im, "point: (%d,%d) " %(pt1,pt2), (10,10), cv2.FONT_HERSHEY_DUPLEX, 0.5, (147, 58, 31), 1)
    #im_with_keypoints = cv.drawKeypoints(im, keypoints[0:1], np.array([]), (255,0,255), 0)
    try:
        b=cv2.drawKeypoints(im, [keypoints[0]], np.array([]), (255,255,0), 0)
        pupil_mid=np.array(keypoints[0].pt)
        im_with_keypoints = cv2.drawKeypoints(im, [center,keypoints[0]], np.array([]), (255,255,0), 0)
    except UnboundLocalError:
        try:
            pupil_mid=np.array(keypoints[0].pt)
            im_with_keypoints = cv2.drawKeypoints(im, [keypoints[0]], np.array([]), (255,255,0), 0)
        except:return im_with_keypoints,pupil_mid
    except IndexError:
        try:
            pupil_mid=np.array(center.pt)
            im_with_keypoints = cv2.drawKeypoints(im, [center], np.array([]), (255,255,0), 0)
        except:return im_with_keypoints,pupil_mid
    return im_with_keypoints,pupil_mid

def checkside(pupil_left,pupil_right,center_left,center_right,thr):
    center_left=np.array(center_left.pt)
    center_right=np.array(center_right.pt)
    dist1=(pupil_left[0]-center_left[0])**3
    dist2=(pupil_right[0]-center_right[0])**3
    if dist1>thr and dist2>thr:
        return 'LEFT'
    elif dist1<-thr and dist2<-thr:
        return 'RIGHT'
    else:
        return 'MIDDLE'

def find_center(t):
    start=time.time()
    center_left=cv2.KeyPoint()
    center_right=cv2.KeyPoint()
    center_left.pt=(0,0)
    center_right.pt=(0,0)
    left_=[]
    right_=[]
    gaze = GazeTracking()
    webcam = cv2.VideoCapture(k)
    i=0
    print('camera detection period (initialize time should be greater than this) :',abs(time.time()-start),'\ninitialize_time :',initialize_time)  
    while abs(time.time()-start)<t:
        # We get a new frame from the webcam
        _, frame = webcam.read()

        # We send this frame to GazeTracking to analyze it
        try:
            gaze.refresh(frame)
        except:print('frame_fucked')
        flag=0
        try:
            left=gaze.eye_left.frame
            right=gaze.eye_right.frame
            try:
                #res_left = cv2.resize(left, (10*left.shape[1],10*left.shape[0])) 
                res_left=left
                res_right=right
            except: print('RESIZE ___')

            flag=1
        except:
            print('pupil detection failed')
            try:
                landmarks = self._predictor(frame, faces[0])
            except:
                #print("landmarks detection failed")
                try:
                    faces= self._face_detector(frame)
                except:print("face_detection failed")
        if flag==1:
            i+=1
            frame1,pupil_left=blob(high(res_left),center_left)
            frame2,pupil_right=blob(high(res_right),center_right)
            frame2 = cv2.resize(frame2, (frame1.shape[1],frame1.shape[0])) 
            frame=np.hstack((high(res_left),high(res_right)))
            left_.append(pupil_left)
            right_.append(pupil_right)
            cv2.putText(frame, 'Calibrating ... ', (30,30), cv2.FONT_HERSHEY_DUPLEX, 0.9, (147, 58, 31), 1)
        else:
            frame=np.zeros((170,170))
        cv2.imshow("Demo", frame)

        if cv2.waitKey(1) == 27:
            break
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    left_=np.array(left_)
    right_=np.array(right_)
    try:
        left_=tuple(np.average(left_,axis=0))
        right_=tuple(np.average(right_,axis=0))
    except:
        print('check initialize_time')
        return 0
    center_left.pt=left_
    center_right.pt=right_
    return center_left,center_right
    

In [ ]:
"""
Demonstration of the GazeTracking library.
Check the README.md for complete documentation.
"""

import cv2
import os, shutil
import numpy as np
from gaze_tracking import GazeTracking

gaze = GazeTracking()
webcam = cv2.VideoCapture(k)
i=0
print('%%%%')
center_left,center_right=find_center(initialize_time)
s=time.time()
while True:
    # We get a new frame from the webcam
    _, frame = webcam.read()

    # We send this frame to GazeTracking to analyze it
    try:
        gaze.refresh(frame)
    except:print('frame_fucked')
    flag=0
    try:
        left=gaze.eye_left.frame
        right=gaze.eye_right.frame
        try:
            #res_left = cv2.resize(left, (10*left.shape[1],10*left.shape[0])) 
            res_left=left
            res_right=right
        except: print('RESIZE ___')
        
        flag=1
    except:
        print('pupil detection failed')
        try:
            landmarks = self._predictor(frame, faces[0])
        except:
            #print("landmarks detection failed")
            try:
                faces= self._face_detector(frame)
            except:print("face_detection failed")
    
    if flag==1:
        i+=1
        cv2.imwrite('E:/ML_IP/ML_projects/Eye_tracking/gaze/eye_frames_2/left/%d__thres_49.jpg'%i,res_left)
        
        frame1,pupil_left=blob(high(res_left),center_left)
        frame2,pupil_right=blob(high(res_right),center_right)
        side=checkside(pupil_left,pupil_right,center_left,center_right,1100)
        frame2 = cv2.resize(frame2, (frame1.shape[1],frame1.shape[0])) 
        frame=np.hstack((frame1,frame2))
        #print(frame.shape)
        cv2.putText(frame, side, (30,30), cv2.FONT_HERSHEY_DUPLEX, 0.9, (147, 58, 31), 1)
        #cv2.imwrite('eye_frames/eyes_test/%d.jpg'%i,frame)
    else:
        frame=np.zeros((170,170))
        #frame=gaze.annotated_frame()

    cv2.imshow("Demo", frame)

    if cv2.waitKey(1) == 27:
        break
    if time.time()-s>40:
        center_left,center_right=find_center(30)
        s=time.time()
        webcam = cv2.VideoCapture(k)

%%%%
camera detection period (initialize time should be greater than this) : 2.607917547225952 
initialize_time : 20
camera detection period (initialize time should be greater than this) : 3.6630728244781494 
initialize_time : 20
